# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import polars as pl
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pl.read_csv(demonstration_fp, parse_dates=True, dtypes={
    "Sex": pl.Categorical,
    "Embarked": pl.Categorical})
df.head()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Ow...","""male""",22,0,7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. ...","""female""",38,0,71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Mis...","""female""",26,0,7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs....","""female""",35,0,53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. Wil...","""male""",35,0,8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


Now, let's check the data types of our DataFrame:

In [4]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': polars.datatypes.Int64,
 'Name': polars.datatypes.Utf8,
 'Sex': polars.datatypes.Categorical,
 'Age': polars.datatypes.Int64,
 'Parch': polars.datatypes.Int64,
 'Fare': polars.datatypes.Float64,
 'Cabin': polars.datatypes.Utf8,
 'Embarked': polars.datatypes.Categorical,
 'Birthday': polars.datatypes.Date,
 'Board time': polars.datatypes.Time,
 'Married since': datetime[μs],
 'all_NA': polars.datatypes.Utf8}

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [5]:
df["Birthday"][0]

datetime.date(1937, 10, 28)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [6]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:127: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [7]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': "<class 'polars.datatypes.Int64'>", 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': "<class 'polars.datatypes.Utf8'>", 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': "<class 'polars.datatypes.Categorical'>", 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': "<class 'polars.datatypes.Int64'>", 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'discrete',

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [8]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [9]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f64
88,"""W#Xj7;<Tw0NB59...","""male""",55,0,47.360353,null,"""S""",1930-08-16,16:00:18,2022-08-13 02:54:45,null
121,"""Wuvhub:""y(;k""","""male""",66,0,56.172302,null,"""S""",1936-09-21,14:32:39,2022-07-20 07:47:27,null
332,"""ze2""*AD *PP;]2...","""male""",78,0,53.49126,null,"""S""",1916-06-06,15:11:59,2022-07-15 14:10:28,null
437,""" *@B>rOZ#Ki6El...","""female""",43,0,1.917574,"""ESFUu ;(w)>Q""","""S""",1904-10-29,15:20:16,2022-08-07 19:13:35,null
235,""" *jO)e.V~ ]f....","""female""",null,0,34.132322,null,"""C""",1913-04-22,11:31:22,2022-08-08 20:59:41,null


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [10]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': "<class 'polars.datatypes.Int64'>",
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [11]:
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f64
1,"""_E lij 1w;1/u:...","""male""",null,0,60.688057,null,"""S""",1938-12-03,16:07:33,2022-08-01 21:16:02,null
2,"""XJ2s`b+n+|(@M(...","""female""",80,0,70.953769,"""G~v/ieIXO""","""S""",1918-05-14,17:19:01,2022-07-27 15:07:08,null
3,"""%""#\/%!Mo_Xi#M...","""female""",43,0,8.513012,"""Gfv:,LfrT-""","""S""",1932-08-17,11:07:51,2022-08-03 08:39:16,null
4,""" #;ehuo2fOUoI=...","""male""",72,0,4.470574,"""BvLz$nhm(q]x""","""S""",1913-01-15,17:58:20,2022-08-01 07:37:59,null
5,"""OVFZPfVc3 Ym.t...","""male""",26,0,4.83751,null,"""S""",1914-01-08,13:21:40,2022-07-16 16:53:20,null


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f64
1,"""Gary Wilkerson...","""male""",28,0,41.684688,null,"""S""",1905-11-02,15:23:42,2022-07-28 01:22:43,null
2,"""Micheal Anders...","""male""",2,1,18.380683,"""G,6 |T#D""","""Q""",1936-02-21,14:44:41,2022-07-20 09:18:30,null
3,"""Nicole Garcia""","""male""",null,2,29.146061,null,"""Q""",1919-01-14,10:47:08,2022-08-10 16:59:18,null
4,"""Monica Fisher""","""male""",19,1,77.23558,null,"""S""",1931-01-22,12:44:43,2022-08-02 16:49:47,null
5,"""Tammy Thompson...","""male""",19,1,106.763376,null,"""S""",1937-12-24,13:07:30,2022-08-12 04:00:37,null


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [13]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,f64,cat,date,time,datetime[μs],f64
1,"""Lisa Velazquez...","""female""",25,1,0.203469,null,"""S""",1926-05-18,13:07:04,2022-08-10 14:45:59,null
2,"""Suzanne Cook""","""male""",null,0,2.728203,null,"""S""",1904-03-20,17:25:56,2022-08-12 17:57:08,null
3,"""Susan Pearson""","""male""",39,0,4.143823,null,"""S""",1911-05-29,11:29:41,2022-08-06 10:44:00,null
4,"""Angela Cunning...","""male""",null,0,4.493274,null,"""S""",null,15:34:29,2022-08-11 22:40:46,null
5,"""Martha Robinso...","""male""",29,1,1.713866,null,"""S""",1926-01-06,10:48:30,2022-08-14 21:34:05,null


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [14]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f64
1,"""Kari Vargas""","""male""",34,1,1.361304,null,"""C""",1939-08-16,15:35:31,null,null
2,"""Laura Parker""","""female""",31,1,0.887823,"""F930""","""C""",1919-10-04,15:39:20,2022-07-27 05:43:56,null
3,"""Charles Pachec...","""female""",29,0,1.708071,null,"""S""",1926-02-06,17:46:08,null,null
4,"""Adam Powell""","""male""",32,0,0.627927,null,"""S""",1935-09-04,14:44:31,null,null
5,"""Elizabeth Gonz...","""female""",null,0,1.378949,null,"""S""",1906-01-15,12:05:32,2022-08-07 07:38:25,null
6,"""Richard Griffi...","""male""",27,2,0.302797,null,"""C""",1913-10-23,18:30:15,2022-07-17 02:47:40,null
7,"""Casey Hurst""","""female""",39,1,2.475183,null,"""S""",1939-01-14,14:07:49,2022-07-17 07:33:22,null
8,"""Lydia Thompson...","""male""",20,0,0.829302,null,"""S""",1927-06-26,16:16:17,2022-07-17 01:34:56,null
9,"""Lori Perez""","""female""",34,0,1.121674,null,"""C""",1930-05-01,17:27:47,2022-08-10 16:11:59,null
